In [1]:
import os

import pandas as pd
import autonormalize as an

In [5]:
crime_df = pd.read_csv(os.path.join(os.getcwd(), 'autonormalize/downloads/crime.csv'))
crime_df = crime_df.drop(columns=crime_df.columns[10:])
crime_df = crime_df.dropna()
crime_df = crime_df.drop_duplicates()

In [4]:
deps_exact = an.find_dependencies(crime_df, 1.00)
deps_approx = an.find_dependencies(crime_df, 0.96)
print("Exact dependencies...")
print(deps_exact)
print("\nApproximate dependencies...")
print(deps_approx)

100%|██████████| 9/9 [11:17<00:00, 64.69s/it]


Exact dependencies...
 --> Record ID
 {Record ID}  --> Agency Code
 {Record ID}  --> Agency Name
 {Record ID}  --> Agency Type
 {Record ID}  --> City
 {Record ID}  {Agency Code}  --> State
 {Record ID}  --> Year
 {Record ID}  --> Month
 {Record ID}  --> Incident
 {Record ID}  --> Crime Type
Approximate dependencies...
 --> Record ID
 {Record ID}  {State,Agency Name}  {Agency Name,City,Agency Type}  --> Agency Code
 {Record ID}  {Agency Code}  --> Agency Name
 {Record ID}  {Agency Code}  {State,Agency Name}  {Agency Name,City}  --> Agency Type
 {Agency Code}  {Record ID}  {Agency Name,Month,Incident}  {State,Agency Name}  {Agency Name,Agency Type}  {Agency Name,Year}  --> City
 {City,Year,Month,Agency Type}  {Agency Code}  {Record ID}  {Agency Name,City}  {Agency Name,Year,Month}  --> State
 {Record ID}  --> Year
 {Record ID}  --> Month
 {Record ID}  --> Incident
 {Agency Type}  {Agency Code}  {Incident}  {Record ID}  {State}  {Year}  {City}  {Agency Name}  {Month}  --> Crime Type


In [6]:
groupings = an.normalize_dependencies(deps_approx)
for grp in groupings:
    print('\n~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~\n')
    print(grp)


~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

 --> Record ID
 {Record ID}  --> Agency Code
 {Record ID}  --> Year
 {Record ID}  --> Month
 {Record ID}  --> Incident

~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

 --> Agency Code
 {Agency Code}  --> Agency Name
 {Agency Code}  --> State

~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

 --> State
 {State}  --> Crime Type

~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

 --> State
 --> Agency Name
 {State,Agency Name}  --> City

~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

 --> Agency Name
 --> City
 {Agency Name,City}  --> Agency Type
